In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from IPython.display import clear_output
import time
import catboost
import re
import optuna

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (FunctionTransformer, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer,
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin

col_names = []
with open('../data/Faults27x7_var','r') as f:
    for line in f:
        col_names.append(line.strip())
        
train_org = pd.read_csv('../data/train.csv')
test_org = pd.read_csv('../data/test.csv')
org_data = pd.read_csv('../data/Faults.NNA', delimiter='\s', engine='python', names=col_names)

X = train_org.drop(['id','Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps','Other_Faults'], axis=1)
pastry = train_org['Pastry'].copy()
z_scratch = train_org['Z_Scratch'].copy()
k_scatch = train_org['K_Scatch'].copy()
stains = train_org['Stains'].copy()
dirtiness = train_org['Dirtiness'].copy()
bumps = train_org['Bumps'].copy()
other_faults = train_org['Other_Faults'].copy()

ys = [pastry, z_scratch, k_scatch, stains, dirtiness, bumps, other_faults]
y_names = ['pastry', 'z_scratch', 'k_scatch', 'stains', 'dirtiness', 'bumps', 'other_faults']

class KMeansTransformer(BaseEstimator,TransformerMixin):
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=self.n_clusters, n_init=10, random_state=0)
        
    def fit(self,X, y=None):
        self.kmeans.fit(X)
        return self
        
    def transform(self,X):
        labels = self.kmeans.predict(X)
        return np.c_[X, labels]
    
class PCA_Transformer(BaseEstimator,TransformerMixin):
    def __init__(self, n_components):
        self.n_components = n_components
        self.pca = PCA(n_components=self.n_components, random_state=0)
        
    def fit(self, X, y=None):
        self.pca.fit(X)
        return self
    
    def transform(self,X):
        cols = self.pca.transform(X)
        return np.c_[X, cols]

In [2]:
y_params_dict = {}
for y_nm in y_names:
    y_params_dict[y_nm] = {}
    for i in range(0,9):
        if i ==1:
            continue
        y_params_dict[y_nm][i] = None

In [3]:
y_params_dict

{'pastry': {0: None,
  2: None,
  3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None},
 'z_scratch': {0: None,
  2: None,
  3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None},
 'k_scatch': {0: None,
  2: None,
  3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None},
 'stains': {0: None,
  2: None,
  3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None},
 'dirtiness': {0: None,
  2: None,
  3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None},
 'bumps': {0: None,
  2: None,
  3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None},
 'other_faults': {0: None,
  2: None,
  3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None}}

In [4]:
y_scores = []

In [5]:
cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for y,yn in zip(ys,y_names):
    for n,i in enumerate(range(1,9)):
        
        if n == 0:
            lgbc = lgb.LGBMClassifier(random_state= 0, objective='binary', verbose=-1)        
            pipe = Pipeline(
                steps = [
                    ('scaler', StandardScaler()),
                    ('model', lgbc)
                ]
            )
            score_without_hpt = np.mean(cross_val_score(pipe, X,y, scoring='roc_auc', cv= cvo))
            
            
            def objective_lgb(trial):
    
                cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
                params = dict(
                    n_estimators = trial.suggest_int('n_estimators',100,500),
                    max_depth = trial.suggest_int('max_depth',2,64),
                    num_leaves = trial.suggest_int('num_leaves',2,128),
                    learning_rate = trial.suggest_float('learning_rate',0.001,0.3),
                    min_child_samples = trial.suggest_int('min_child_samples',2,500),
                    min_child_weight = trial.suggest_float('min_child_weight', 0.01,10),
                    subsample = trial.suggest_float('subsample', 0.33,0.85),
                    colsample_bytree = trial.suggest_float('colsample_bylevel',0.33,0.7),
                    reg_alpha=trial.suggest_float('reg_alpha', 0.001, 0.1),
                    reg_lambda = trial.suggest_float('reg_lambda', 0.001,0.1)
                )
                
                lgbc = lgb.LGBMClassifier(random_state= 0, objective='binary', verbose=-1,**params)
                
                pipe = Pipeline(
                    steps = [
                        ('scaler', StandardScaler()),
                        ('model', lgbc)
                    ]
                )
                
                score = np.mean(cross_val_score(pipe, X,y, scoring='roc_auc', cv= cvo))
                return score
            
            study_lgb = optuna.create_study(direction='maximize')
            
            study_lgb.optimize(objective_lgb, n_trials=100, n_jobs=-1, show_progress_bar=True)
            
            best_params = study_lgb.best_params
            y_params_dict[yn][0] = best_params
            
            best_score = study_lgb.best_value
            y_scores.append((yn,f'score_without_hpt: {score_without_hpt}', f'score_after_hpt: {best_score}'))
            
            time.sleep(5)
            clear_output()
            continue
            
        lgbc = lgb.LGBMClassifier(random_state= 0, objective='binary', verbose=-1)        
        pipe = Pipeline(
            steps = [
                ('scaler', StandardScaler()),
                ('kmeans', KMeansTransformer(n_clusters=i)),
                ('model', lgbc)
            ]
        )
        score_without_hpt = np.mean(cross_val_score(pipe, X,y, scoring='roc_auc', cv= cvo))
        
        def objective_lgb(trial):
    
            cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
            params = dict(
                n_estimators = trial.suggest_int('n_estimators',100,500),
                max_depth = trial.suggest_int('max_depth',2,64),
                num_leaves = trial.suggest_int('num_leaves',2,128),
                learning_rate = trial.suggest_float('learning_rate',0.001,0.3),
                min_child_samples = trial.suggest_int('min_child_samples',2,500),
                min_child_weight = trial.suggest_float('min_child_weight', 0.01,10),
                subsample = trial.suggest_float('subsample', 0.33,0.85),
                colsample_bytree = trial.suggest_float('colsample_bylevel',0.33,0.7),
                reg_alpha=trial.suggest_float('reg_alpha', 0.001, 0.1),
                reg_lambda = trial.suggest_float('reg_lambda', 0.001,0.1)
            )
            
            lgbc = lgb.LGBMClassifier(random_state= 0, objective='binary', verbose=-1,**params)
            
            pipe = Pipeline(
                steps = [
                    ('scaler', StandardScaler()),
                    ('kmeans', KMeansTransformer(n_clusters=i)),
                    ('model', lgbc)
                ]
            )
            
            score = np.mean(cross_val_score(pipe, X,y, scoring='roc_auc', cv= cvo))
            return score
        
        study_lgb = optuna.create_study(direction='maximize')
        
        study_lgb.optimize(objective_lgb, n_trials=100, n_jobs=-1, show_progress_bar=True)
        
        best_params = study_lgb.best_params
        y_params_dict[yn][i] = best_params
        
        best_score = study_lgb.best_value
        y_scores.append((yn,f'score_without_hpt(n_clusters={i}): {score_without_hpt}', f'score_after_hpt(n_clusters={i}): {best_score}'))
        
        time.sleep(5)
        clear_output()
        
        with open('results.txt', 'w') as file:
            for tup in y_scores:
                file.write(f'{tup[0]}, {tup[1]}, {tup[2]}\n')